In [ ]:
import pormake as pm
import numpy as np
import csv

db = pm.Database()
loc = pm.Locator()

## organic nodes
TP_O = pm.BuildingBlock(bb_file="pi-d_building_blocks/TP_O.xyz")
PcM_O = pm.BuildingBlock(bb_file="pi-d_building_blocks/PcM_O.xyz")

## edges
Fe_pln = pm.BuildingBlock(bb_file="pi-d_building_blocks/Fe_pln.xyz")

pm.log.disable_print()
pm.log.disable_file_print()


## Prepare topologies for screening

In [ ]:
def expand_topo(topo):
    x = 1
    y = 1
    z = 1
    edges = topo.edge_indices
    neigh = topo.neighbor_list
    for i in edges:

        ## below set lists unique neighbors for the edges. If 1, this means expansion is required.
        if len(set([j.index for j in neigh[i]])) == 1:
            dist = [abs(d) for d in neigh[i][0].distance_vector]

            ## expand topology along the "major" edge direction to allow for alternating node sequence.
            if x == 1 and dist.index(max(dist)) == 0:
                x = 2
            if y == 1 and dist.index(max(dist)) == 1:
                y = 2
            if z == 1 and dist.index(max(dist)) == 2:
                z = 2

    return topo * (x, y, z)

In [ ]:
## prep topologies for patterned node assignment

topo_codes = []
file = open('cn34_pid_topology_list.csv')
csvreader = csv.reader(file)

for row in csvreader:
    for topo in row:
        topo_codes.append(topo)

topos = []

for topo in topo_codes:
    cur_topo = db.get_topo(topo)
    cur_topo = expand_topo(cur_topo)

    if cur_topo.n_nodes % 2 == 1:
        ## expand in one of the directions to ensure even number of nodes
        cur_topo = cur_topo * (1, 1, 2)
        
    topos.append(cur_topo)

In [ ]:
## check rmsd and topology scale here (check # nodes again since expansion took place)

node_limit = 80
refined_topos = []

for topo in topos:
    
    if topo.n_nodes > node_limit:
        continue
    refined_topos.append(topo)


## Perform screening (tetrahedral connection allowed)

In [ ]:
## attempt generation!
test_builder = pm.Builder(planarity_enforcement=True, angle_threshold=15, check_tetrahedral=True)

successful_cases = []

for topo in refined_topos:
           
    current_node = {}
    current_edge = {}    
    
    ## initialize nodes
    for i, node in enumerate(topo.unique_cn):
        
        if node == 3:
            current_node[i] = TP_O
        elif node == 4:
            current_node[i] = PcM_O
        else:
            print('something wrong with', topo.name)

    
    ## initialize edges
    for i, edge in enumerate(topo.unique_edge_types):
        current_edge[tuple(edge)] = Fe_pln

    bbs = test_builder.make_bbs_by_type(topo, current_node, current_edge)
    calc_permutation = test_builder.extract_permutation(topo, current_node, current_edge)
    calc_permutation = {i : perm for i, perm in enumerate(calc_permutation)}
    
    try:
        mof = test_builder.build(topo, bbs, calc_permutation)

        ## check and eliminate 2D MOFs
        if min(mof.atoms.cell.cellpar()[:3]) < 3:
            continue

        mof.view()
        print(topo.name)
        successful_cases.append(topo.name)

    except:
        continue


## Raw screening results

In [ ]:
successful_cases = ['bpc',
 'cdl',
 'cdm',
 'cdn',
 'cds',
 'cds-b',
 'cdz',
 'clh',
 'etc',
 'hof',
 'lwg',
 'mot',
 'mou',
 'nbo',
 'nta',
 'ssd',
 'sse',
 'ssf',
 'sst',
 'tfc',
 'tfe',
 'tfn',
 'tfz',
 'ths',
 'tta']

## Inspection of raw screening results

Structures from screening are visualized to qualitatively assess their synthesizability.

In [ ]:
## attempt generation!
test_builder = pm.Builder(planarity_enforcement=True, angle_threshold=15, check_tetrahedral=True)

for topo in successful_cases:

    topo = db.get_topo(topo)
           
    current_node = {}
    current_edge = {}    
    
    ## initialize nodes
    for i, node in enumerate(topo.unique_cn):
        
        if node == 3:
            current_node[i] = TP_O
        elif node == 4:
            current_node[i] = PcM_O
        else:
            print('something wrong with', topo.name)

    
    ## initialize edges
    for i, edge in enumerate(topo.unique_edge_types):
        current_edge[tuple(edge)] = Fe_pln

    bbs = test_builder.make_bbs_by_type(topo, current_node, current_edge)
    calc_permutation = test_builder.extract_permutation(topo, current_node, current_edge)
    calc_permutation = {i : perm for i, perm in enumerate(calc_permutation)}
    
    try:
        mof = test_builder.build(topo, bbs, calc_permutation)

        ## check and eliminate 2D MOFs
        if min(mof.atoms.cell.cellpar()[:3]) < 3:
            continue

        mof.view()
        print(topo.name)
        successful_cases.append(topo.name)

    except:
        continue


In [ ]:
keep_cases = [ 'cdl',
 'cdm',
 'cdn',
 'cds',
 'cds-b',
 'cdz',
 'clh',
 'etc',
 'hof',
 'lwg',
 'mot',
 'mou',
 'nbo',
 'nta',
 'ssd',
 'sse',
 'ssf',
 'sst',
 'tfc',
 'tfe',
 'tfn',
 'tfz',
 'ths',
 'tta']